In [ ]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory


df = pd.read_excel('colors_147.xlsx', index_col='MODELS')
total_demand = df.sum(axis=1) 

demand_distribution = total_demand.to_dict()

car_list = []
for model, count in demand_distribution.items():
    car_list.extend([model] * count)


sequence_df = pd.DataFrame({
    'J':range(1,len(car_list) + 1),
    'MODELS': car_list
})


color_list = []
for model in list(sequence_df['MODELS'].unique()):
    
    color_dist = df.loc[model].to_dict()
    colors = list(color_dist.keys())
    for i in colors:
        for j in range(color_dist[i]):
            color_list.append(i)



col_sequence_df = pd.DataFrame({
    'J': range(1, len(car_list) + 1),
    'COLORS': color_list
})

cj = col_sequence_df.set_index('J')['COLORS'].to_dict()
H=13
process_times_df = pd.read_csv('processing_times_of_models.csv').drop(columns='Unnamed: 0') 
#H=1
#process_times_df = pd.read_csv('processing_times_of_models.csv', usecols=['MODELS','EC OVEN','NVH-CEILING ROBOT', 'SEALER'])
#H=4 
#process_times_df = pd.read_csv('processing_times_of_models.csv', usecols=['MODELS','UNDERCOAT ROBOT','UNDERCOAT OVEN', 'UNDERCOAT EMERY','BASECOAT PAINT', 'BASECOAT 1ST STATION']) 
process_times_df.set_index('MODELS', inplace=True)

num_vehicles = df.sum(axis=1).sum()
num_time_slots = df.sum(axis=1).sum()
num_stations = process_times_df.shape[1]
delta_H = 20
#-------------------------------------------------------------------------------------------------------
model = ConcreteModel()
model.J = RangeSet(1, num_vehicles)  # Vehicles
model.T = RangeSet(1, num_time_slots)  # Time slots
model.K = RangeSet(1, num_stations)  # Stations
pjk = {(j, k): process_times_df.at[sequence_df.at[j-1, 'MODELS'],  process_times_df.columns[k-1]]
       for j in model.J for k in model.K}

model.pjk = Param(model.J, model.K, initialize=pjk)
model.cj = Param(model.J, initialize=cj)

# Variables
model.x = Var(model.J, model.T, domain=Binary)
model.C = Var(model.T, model.K, domain=NonNegativeReals)
model.s = Var(model.T, domain=Binary)


def total_completion_time(model):
    return sum(model.C[t, k] for t in model.T for k in model.K)

# Objective function: Minimize the completion time at the last station
#model.obj = Objective(rule=total_completion_time, sense=minimize)
model.obj = Objective(expr=model.C[num_time_slots, num_stations],  sense=minimize)

# Constraints
# Assignment constraints
def assignment1_rule(model, j):
    return sum(model.x[j, t] for t in model.T) == 1
model.assignment1 = Constraint(model.J, rule=assignment1_rule)

def assignment2_rule(model, t):
    return sum(model.x[j, t] for j in model.J) == 1
model.assignment2 = Constraint(model.T, rule=assignment2_rule)

# Completion time constraints
def completion1_rule(model, t, k):
    if t > 1:
        return model.C[t, k] >= model.C[t-1, k] + sum(model.pjk[j, k]  * model.x[j, t] for j in model.J)
    else:
        return model.C[t, k] >= sum(model.pjk[j, k]  * model.x[j, t] for j in model.J)
        
model.completion1 = Constraint(model.T, model.K, rule=completion1_rule)

def completion2_rule(model, t, k):
    if k > 1:
        return model.C[t, k] >= model.C[t, k-1] + sum(model.pjk[j, k]  * model.x[j, t] for j in model.J)
    else:
        return model.C[t, k] >= sum(model.pjk[j, k]  * model.x[j, t] for j in model.J)
model.completion2 = Constraint(model.T, model.K, rule=completion2_rule)

 



# Setup time constraints
def setup_rule(model, t):
    if t > 1:
        return model.C[t, H] >= model.C[t-1, H] + sum(model.pjk[j, H] * model.x[j, t] for j in model.J) + delta_H * model.s[t]
    else:
        return model.C[t, H] == model.C[t, H-1] + sum(model.pjk[j, H] * model.x[j, t] for j in model.J) # Constraint.Skip 
        #return Constraint.Skip
model.setup = Constraint(model.T, rule=setup_rule)


def setup_indicator_rule(model, t, j, j_prime):
    if t > 1 and cj[j] != cj[j_prime] and j != j_prime:  # j != j_prime eklendi
        return model.s[t] >= model.x[j, t] +  model.x[j_prime, t - 1] - 1
    return Constraint.Skip
model.SetupIndicator = Constraint(model.T, model.J, model.J, rule=setup_indicator_rule)





In [ ]:
# Solve the model
print('Started...')
solver = SolverFactory('gurobi')
solver.options['nodefilestart'] = 0.5
#solver.options['threads'] = 2
solver.options['mipgap'] = 0
#solver.options['timelimit'] = 4200
results = solver.solve(model, tee=True)


In [ ]:
if results.solver.termination_condition == TerminationCondition.optimal:
    print("Optimal solution found!")
    model.display()
elif results.solver.termination_condition == TerminationCondition.infeasible:
    print("Infeasible solution")
else:
    print("Solver terminated with condition: ", results.solver.termination_condition)

In [ ]:
rez = []
for t in range(1,len(car_list)+1):
    for j in range(1,len(car_list)+1):
        if value(model.x[j,t]) == 1:
            rez.append(sequence_df.iloc[j-1].values[1])

rezz = pd.DataFrame(rez)

color_output = []
for t in range(1,len(car_list)+1):
    for j in range(1,len(car_list)+1):
        if value(model.x[j,t]) == 1:
            color_output.append(col_sequence_df.iloc[j-1].values[1])

color_output_df = pd.DataFrame(color_output)
final_output = pd.concat([rezz, color_output_df], axis=1)
final_output.to_excel('output.xlsx', index=True)
final_output